## Testing igzip - a recommendation from the gztool repo

In [ ]:
import requests

url = 'https://lifewise.sapphiremrfhub.com/mrfs/202505/MPNMRF_MPI_20250324-MPI_MPI_innetworkrates_FAC_2025-03-21.json.gz'
local_filename = 'MPNMRF_MPI_20250324-MPI_MPI_innetworkrates_FAC_2025-03-21.json.gz'

# Downloads the file: streams directly from get request to file - still compressed though
with requests.get(url, stream=True) as r:
    r.raise_for_status()
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

import os

# Get the size of the downloaded file
file_size = os.path.getsize(local_filename)

# Print the size of the file in bytes
print(f"Size of the file: {file_size} bytes")


In [ ]:
# After downloaded, move to the test-data directory (I also made a script to do this in init_scripts called download_test_gzip.sh)
import os
os.rename(local_filename, '../../test-data/lifewise/secondary-gzipped/test-file.gz')

In [32]:
# Now we can use the igzip library to build the index
import indexed_gzip as igzip

# Open the gzip file with indexed_gzip
with igzip.IndexedGzipFile(local_filename, drop_handles=False) as f:
    # Builds the index by reading through the file
    while f.read(1024 * 1024):
        pass
    # Saves the index to a file for our future use
    f.export_index('file_index.pkl')


In [38]:
import indexed_gzip as igzip

# Open the gzip file
with igzip.IndexedGzipFile(local_filename, drop_handles=False) as f:
    # Load the previously saved index
    f.import_index('file_index.pkl')
    
    # Seek to a specific offset in the uncompressed data
    f.seek(5005000)  # For example, seek to byte 1,000,000
    data = f.read(100)  # Read 100 bytes from that position
    print(data)


b'12-31","negotiated_type":"derived","billing_class":"institutional","billing_code_modifier":["CQ"],"a'
